<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h1 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<span dir="ltr" style="unicode-bidi: isolate;">M5.3</span> – إعداد سير عمل معالجة البيانات
</h1>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
جزء من:
<a href="https://github.com/OpenClimateScience/M5-Open-Science-for-Crops" dir="ltr">
علوم المناخ المفتوحة للمحاصيل وظروفها
</a>
</p>

</div>


In [ ]:
import datetime
import glob
import os
import earthaccess
import numpy as np
import h5py
import xarray as xr
import rasterio
import rioxarray
import py4eos
import pyproj
from shapely.geometry import Polygon
from rasterio.warp import calculate_default_transform, reproject, Resampling
from matplotlib import pyplot
from tqdm import tqdm

auth = earthaccess.login()

LC_DIR = 'data/MCD12Q1'
VNP16_DIR = 'data/VNP16A2'
IMERG_DIR = 'data/IMERG'
OUTPUT_LC_FILENAME = 'data/processed/MODIS_MCD12Q1_Type5_cereal_croplands_h18v05_2023.tiff'
OUTPUT_VNP16_DIR = 'data/processed/VNP16_ET_and_PET'
OUTPUT_IMERG_DIR = 'data/processed'
TIME_PERIOD = ('2023-10-01', '2024-09-30')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
نظرة عامة
</h2>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لحساب مؤشر رضا متطلبات المياه 
<span dir="ltr" style="unicode-bidi: isolate;">WRSI</span>، 
سنحتاج إلى البيانات التالية:
</p>

<ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
خريطة للغطاء الأرضي لمساعدتنا على تحديد مواقع الأراضي الزراعية في منطقة الدراسة.
</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
النتح-التبخر المحتمل 
<span dir="ltr" style="unicode-bidi: isolate;">PET</span>.
</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
الهطول.
</li>
</ul>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لكل عنصر من هذه العناصر، لدى 
<span dir="ltr" style="unicode-bidi: isolate;">NASA</span> 
منتج يمكننا استخدامه:
</p>

<ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
مجموعة بيانات 
<span dir="ltr" style="unicode-bidi: isolate;">MODIS MCD12Q1</span> 
(<a href="https://doi.org/10.5067/MODIS/MCD12Q1.061" dir="ltr">الرابط</a>) 
تصف فئة الغطاء الأرضي لكل بكسل بدقة 
<span dir="ltr" style="unicode-bidi: isolate;">500</span> 
متر على شبكة 
<span dir="ltr" style="unicode-bidi: isolate;">MODIS</span> 
الجيبية 
<span dir="ltr" style="unicode-bidi: isolate;">(sinusoidal)</span>.
</li>

<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
كما رأينا سابقًا، يوفر منتج 
<span dir="ltr" style="unicode-bidi: isolate;">MODIS MOD16A2</span> 
(<a href="https://doi.org/10.5067/MODIS/MOD16A2.006" dir="ltr">الرابط</a>) 
تقديرات النتح-التبخر الأرضي 
<span dir="ltr" style="unicode-bidi: isolate;">ET</span> 
والنتح-التبخر المحتمل 
<span dir="ltr" style="unicode-bidi: isolate;">PET</span> 
كل 
<span dir="ltr" style="unicode-bidi: isolate;">8</span> 
أيام على مستوى العالم.
</li>

<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
واستخدمنا أيضًا سابقًا بيانات الهطول من 
<span dir="ltr" style="unicode-bidi: isolate;">NASA</span> 
ضمن مجموعة بيانات 
<span dir="ltr" style="unicode-bidi: isolate;">IMERG-Final</span> 
التابعة لمهمة 
<span dir="ltr" style="unicode-bidi: isolate;">Global Precipitation Monitoring (GPM)</span> 
(<a href="https://disc.gsfc.nasa.gov/datasets/GPM_3IMERGDF_06/summary" dir="ltr">الرابط</a>).
</li>
</ul>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>
في هذا الدرس سنجهّز مجموعات البيانات اللازمة لحساب 
<span dir="ltr" style="unicode-bidi: isolate;">WRSI</span>. 
ومن المهم أيضًا أننا سنرى كيف نكتب قواعد 
<span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> 
التي ستمكّن المستخدمين لاحقًا من إعادة إنتاج خطوات معالجة البيانات بسرعة.
</strong>
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
على وجه التحديد، نريد أن نقوم بما يلي:
</p>

<ol dir="rtl" style="direction: rtl; unicode-bidi: plaintext; text-align: right; padding-right: 1.2em;">
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
إعادة أخذ العينات وإسقاط بيانات الغطاء الأرضي على شبكة 
<span dir="ltr" style="unicode-bidi: isolate;">EASE-Grid 2.0</span> 
بدقة 
<span dir="ltr" style="unicode-bidi: isolate;">1</span> 
كم، لتسهيل مقارنتها مع مجموعات بيانات أخرى.
</li>
</ol>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<hr>

<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
الحصول على البيانات التي نحتاجها
</h2>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
سنواصل استخدام مكتبة 
<span dir="ltr" style="unicode-bidi: isolate;">earthaccess</span> 
للوصول البرمجي إلى 
<span dir="ltr" style="unicode-bidi: isolate;">NASA Earthdata Search</span>. 
تذكّر أن الدالة 
<span dir="ltr" style="unicode-bidi: isolate;">earthaccess.search_data()</span> 
تتضمن وسيطًا باسم 
<span dir="ltr" style="unicode-bidi: isolate;">bounding_box</span> 
يتيح لنا البحث عن مجموعات البيانات التي تقع ضمن مستطيل إحداثي مُعرَّف بإحداثيات خطوط الطول ودوائر العرض.
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>help(earthaccess.search_data)</code></pre>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>**bounding_box**: a tuple representing spatial bounds in the form
    `(lower_left_lon, lower_left_lat, upper_right_lon, upper_right_lat)`</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
سنعرّف صندوق الإحاطة الخاص بنا 
<span dir="ltr" style="unicode-bidi: isolate;">bbox</span> 
لمنطقة الدراسة، وهي جزء من شمال أفريقيا يتمركز حول الجزائر.
</p>

</div>


In [ ]:
# The bounding box for our study area
bbox = (1.5, 34.0, 8.0, 37.0)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تنزيل خريطة الغطاء الأرضي لمنطقة الدراسة
</h3>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
عند بدء تنزيل البيانات التي سنستخدمها، ينبغي أن نتذكّر تسجيل 
<strong>المعرّف الرقمي للكائن (Digital Object Identifier – DOI)</strong> 
لكل مجموعة بيانات. لا تتيح لنا مُعرّفات 
<span dir="ltr" style="unicode-bidi: isolate;">DOI</span> 
الاستشهاد الصحيح بالبيانات ومنح مؤلفيها حقهم العلمي فحسب، بل تمكّن الآخرين أيضًا من تحديد مجموعة البيانات التي استخدمناها بدقة.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
بالنسبة لبيانات الغطاء الأرضي 
<span dir="ltr" style="unicode-bidi: isolate;">MODIS MCD12Q1</span>، 
فإن المعرّف الرقمي هو:
</p>

<ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<span dir="ltr" style="unicode-bidi: isolate;">DOI:</span>
<a href="https://doi.org/10.5067/MODIS/MCD12Q1.061" dir="ltr">
https://doi.org/10.5067/MODIS/MCD12Q1.061
</a>
</li>
</ul>

</div>


In [ ]:
# Note that TIME_PERIOD refers to one of our global variables, defined at the top of the script
results = earthaccess.search_data(
    short_name = 'MCD12Q1',
    temporal = TIME_PERIOD,
    bounding_box = tuple(bbox))

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يوجد نتيجتان لأن 
<span dir="ltr" style="unicode-bidi: isolate;">TIME_PERIOD</span> 
يمتد عبر سنتين.
</p>

</div>


In [ ]:
len(results)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
عند البدء في التفكير في إنشاء خط أنابيب لمعالجة البيانات قابل لإعادة الإنتاج، من الأمور التي قد نرغب في تجنّبها إعادة تنزيل مجموعات بيانات كبيرة في كل مرة. إذا كانت البيانات موجودة بالفعل، فلا حاجة لتنزيلها مرة أخرى.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
نظرًا لأننا حرصنا على إنشاء مجلدات منفصلة لكل مجموعة بيانات، يمكننا إجراء فحص بسيط للتحقق من وجود الملفات لتحديد ما إذا كنا بحاجة إلى تنزيل أي بيانات إضافية. تذكّر أن الدالة 
<span dir="ltr" style="unicode-bidi: isolate;">glob.glob()</span> 
تعرض محتويات المسار الذي نبحث فيه، حيث يُستخدم الرمز 
<span dir="ltr" style="unicode-bidi: isolate;">*</span> 
كحرف بديل يطابق أي تسلسل من الأحرف في اسم الملف:
</p>

</div>


In [ ]:
# This is the directory we're searching
print(f'{LC_DIR}/*')

In [ ]:
# Only download the files once; i.e., if we haven't already downloaded any
if len(glob.glob(f'{LC_DIR}/*')) == 0:
    earthaccess.download(results, LC_DIR)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>
سنستخدم مكتبة 
<span dir="ltr" style="unicode-bidi: isolate;">py4eos</span>، 
التي اطّلعنا عليها سابقًا في الوحدة 
<span dir="ltr" style="unicode-bidi: isolate;">3</span> 
(<a href="https://github.com/OpenClimateScience/M3-Open-Science-for-Water-Resources" dir="ltr">الرابط</a>)، 
لفتح مجموعة بيانات الغطاء الأرضي 
<span dir="ltr" style="unicode-bidi: isolate;">MODIS MCD12Q1</span> 
لسنة 
<span dir="ltr" style="unicode-bidi: isolate;">2023</span>.
</strong>
</p>

</div>


In [ ]:
hdf = py4eos.read_file('data/MCD12Q1/MCD12Q1.A2023001.h18v05.061.2024252125305.hdf', platform = 'MODIS')
hdf

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
بعد ذلك، نريد الحصول على مجموعة بيانات الغطاء الأرضي ككائن من نوع 
<span dir="ltr" style="unicode-bidi: isolate;">rasterio</span>. 
سنستخدم الطبقة 
<span dir="ltr" style="unicode-bidi: isolate;">"LC_Type5"</span>، 
وهي واحدة من عدة تصنيفات للغطاء الأرضي المتاحة ضمن مجموعة بيانات 
<span dir="ltr" style="unicode-bidi: isolate;">MODIS MCD12Q1</span>.
</p>

</div>


In [ ]:
lc_raster = hdf.to_rasterio('LC_Type5', filename = '', driver = 'MEM')

In [ ]:
lc_map = lc_raster.read(1)

# Plot a preview
pyplot.imshow(lc_map, interpolation = 'nearest', vmax = 11)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
إسقاط بيانات الغطاء الأرضي
</h3>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
بعد ذلك، نريد إسقاط بيانات الغطاء الأرضي على شبكة 
<span dir="ltr" style="unicode-bidi: isolate;">EASE-Grid 2.0</span> 
بدقة 
<span dir="ltr" style="unicode-bidi: isolate;">1</span> 
كم. قد يكون من الصعب حساب العرض 
<span dir="ltr" style="unicode-bidi: isolate;">width</span> 
والارتفاع 
<span dir="ltr" style="unicode-bidi: isolate;">height</span> 
الجديدين لبيانات نقطية بعد الإسقاط، لذلك نستخدم الدالة 
<a href="https://rasterio.readthedocs.io/en/latest/api/rasterio.warp.html#rasterio.warp.calculate_default_transform" dir="ltr">
calculate_default_transform()
</a> 
من الوحدة 
<span dir="ltr" style="unicode-bidi: isolate;">rasterio.warp</span> 
لتحديد هذه القيم.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
في 
<span dir="ltr" style="unicode-bidi: isolate;">calculate_default_transform()</span> 
نحتاج إلى تزويدها بنظام الإحداثيات المرجعي 
<span dir="ltr" style="unicode-bidi: isolate;">(CRS)</span> 
لبياناتنا، ونظام الإحداثيات الجديد الذي نريده، والعرض والارتفاع الحاليين، وحدود الامتداد المكاني للرستر (المعطاة بواسطة 
<span dir="ltr" style="unicode-bidi: isolate;">lc_raster.bounds</span>). 
مخرجات هذه الدالة عبارة عن ثلاثية 
<span dir="ltr" style="unicode-bidi: isolate;">(3-element tuple)</span>، 
لذلك يمكننا إسناد الناتج إلى ثلاثة متغيرات. المتغيران الأخيران يمثلان القيم الجديدة لـ 
<span dir="ltr" style="unicode-bidi: isolate;">width</span> 
و<span dir="ltr" style="unicode-bidi: isolate;">height</span> 
للرستر المُسقَط. أما المخرج الأول، 
<span dir="ltr" style="unicode-bidi: isolate;">new_transform</span>، 
فيحتوي على معلومات إضافية مهمة حول الرستر الجديد الذي سننشئه بعد الإسقاط.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F449; لاحظ أن البلاطة النموذجية لبيانات 
<span dir="ltr" style="unicode-bidi: isolate;">MODIS</span> 
بدقة 
<span dir="ltr" style="unicode-bidi: isolate;">500</span> 
متر تتكوّن من 
<span dir="ltr" style="unicode-bidi: isolate;">2400</span> 
صف و
<span dir="ltr" style="unicode-bidi: isolate;">2400</span> 
عمود. لذلك، إذا قمنا بإعادة أخذ العينات إلى دقة 
<span dir="ltr" style="unicode-bidi: isolate;">1</span> 
كم (أي بكسلات أكبر بمرتين)، فينبغي أن يكون عدد الصفوف والأعمدة نصف العدد الأصلي. أي إن شكل المصفوفة الناتجة يجب أن يكون 
<span dir="ltr" style="unicode-bidi: isolate;">(1200, 1200)</span>. 
في المثال أدناه، هذا هو الشكل الذي نزوده للدالة 
<span dir="ltr" style="unicode-bidi: isolate;">calculate_default_transform()</span>.
</p>

</div>


In [ ]:
# NOTE: 6933 is the EPSG code for the global EASE-Grid 2.0 projection
new_transform, width, height = calculate_default_transform(
    lc_raster.crs, pyproj.CRS(6933), 1200, 1200, *lc_raster.bounds)

new_transform

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
أصبحنا الآن جاهزين لإسقاط رستر الغطاء الأرضي باستخدام الدالة 
<span dir="ltr" style="unicode-bidi: isolate;">project()</span> 
من الوحدة 
<span dir="ltr" style="unicode-bidi: isolate;">rasterio.warp</span>. 
هناك عدة نقاط ينبغي الانتباه إليها في الشيفرة المستخدمة:
</p>

<ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
نحتاج إلى فتح ملف إخراج لكتابة البيانات إليه؛ نستخدم لذلك الدالة 
<span dir="ltr" style="unicode-bidi: isolate;">rasterio.open()</span> 
مع وضع الملف 
<span dir="ltr" style="unicode-bidi: isolate;">'w+'</span> 
بحيث يمكننا كتابة البيانات وقراءتها.
</li>

<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
نوع البيانات 
<span dir="ltr" style="unicode-bidi: isolate;">dtype</span> 
المستخدم هو 
<span dir="ltr" style="unicode-bidi: isolate;">np.uint8</span>، 
وهو عدد صحيح غير موقّع بدقة 
<span dir="ltr" style="unicode-bidi: isolate;">8</span> 
بت.
</li>

<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
استخدمنا 
<span dir="ltr" style="unicode-bidi: isolate;">Resampling.mode</span> 
(أي إعادة أخذ العينات بطريقة الأغلبية) لأن القيم في رستر الغطاء الأرضي تمثل فئات تصنيفية، ولا ينبغي استيفاؤها عدديًا كما لو كانت قيَمًا مستمرة.
</li>

<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
ملف الإخراج 
<span dir="ltr" style="unicode-bidi: isolate;">OUTPUT_LC_FILENAME</span> 
المعرّف في أعلى هذا الدفتر سيكون بصيغة 
<span dir="ltr" style="unicode-bidi: isolate;">GeoTIFF</span>، 
لأننا استخدمنا الامتداد 
<span dir="ltr" style="unicode-bidi: isolate;">.tiff</span>.
</li>
</ul>

</div>


In [ ]:
# See more at: https://rasterio.readthedocs.io/en/stable/topics/resampling.html

output_raster = rasterio.open(
    OUTPUT_LC_FILENAME, 'w+', count = 1, width = width, height = height,
    dtype = np.uint8, crs = pyproj.CRS(6933), transform = new_transform)

# Writing the new array to the file
reproject(
    source = rasterio.band(lc_raster, 1),
    destination = rasterio.band(output_raster, 1),
    resampling = Resampling.mode,
    src_nodata = 255,
    dst_nodata = 255)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
أخيرًا، نقوم بتحويل خريطة الغطاء الأرضي إلى خريطة ثنائية للأراضي الزراعية، حيث تشير القيمة 
<span dir="ltr" style="unicode-bidi: isolate;">1</span> 
إلى وجود أراضٍ زراعية، وتشير القيمة 
<span dir="ltr" style="unicode-bidi: isolate;">0</span> 
إلى عدم وجودها.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
وفقًا لـ 
<a href="https://lpdaac.usgs.gov/documents/1409/MCD12_User_Guide_V61.pdf" dir="ltr">
دليل مستخدم MODIS MCD12Q1
</a>، 
فإن فئة الأراضي الزراعية في تصنيف 
<span dir="ltr" style="unicode-bidi: isolate;">"LC_Type5"</span> 
مُمثَّلة بالقيمة العددية 
<span dir="ltr" style="unicode-bidi: isolate;">7</span>.
</p>

</div>


In [ ]:
lc_map_1km = output_raster.read(1)

# Create a binary array where Croplands==1, 0 for everything else
croplands = np.where(lc_map_1km == 7, 1, 0)
pyplot.imshow(croplands, interpolation = 'nearest')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
أخيرًا، سنقوم بالكتابة فوق محتويات ملف 
<span dir="ltr" style="unicode-bidi: isolate;">GeoTIFF</span> 
الناتج الذي أنشأناه للتو، بحيث نستبدل خريطة الغطاء الأرضي بخريطة الأراضي الزراعية الجديدة. يمكننا القيام بذلك لأن نوع البيانات المطلوب 
<span dir="ltr" style="unicode-bidi: isolate;">(dtype)</span> 
وكذلك الخصائص المكانية للرستر متطابقة في الحالتين.
</p>

</div>


In [ ]:
output_raster.write(croplands, 1)
output_raster.close()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<hr>

<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
أول ملف <span dir="ltr" style="unicode-bidi: isolate;">Snakefile</span> لنا
</h2>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
بعد أن حدّدنا كيفية معالجة بيانات الغطاء الأرضي، لنستخدم 
<span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> 
لإنشاء قاعدة تتيح لنا تنفيذ هذه العملية بسهولة في المستقبل.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لننشئ ملفًا باسم 
<span dir="ltr" style="unicode-bidi: isolate;">Snakefile</span> 
يتضمن القاعدة التالية:
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>rule download_data:
    params:
        dir_mcd12q1="data/MCD12Q1"
    script:
        "workflows/download_data.py"</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يمثل هذا وصفًا عالي المستوى لأول مهمة لدينا: تنزيل بيانات الغطاء الأرضي. سابقًا استخدمنا الكلمة المفتاحية 
<span dir="ltr" style="unicode-bidi: isolate;">shell</span> 
في قاعدة 
<span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> 
لتنفيذ أوامر سطر أوامر عامة. بدلاً من 
<span dir="ltr" style="unicode-bidi: isolate;">shell</span>، 
نستخدم هنا الكلمة المفتاحية 
<span dir="ltr" style="unicode-bidi: isolate;">script</span> 
للإشارة إلى أن 
<span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> 
سيشغّل سكربت 
<span dir="ltr" style="unicode-bidi: isolate;">Python</span> 
عند تنفيذ القاعدة. ويعرف 
<span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> 
أن الملف سكربت 
<span dir="ltr" style="unicode-bidi: isolate;">Python</span> 
من خلال امتداد الملف.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F449; <strong>لاحظ أن مسارات الملفات نسبية إلى موقع ملف 
<span dir="ltr" style="unicode-bidi: isolate;">Snakefile</span>.</strong>
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F449; <strong>لاحظ الكلمة المفتاحية 
<span dir="ltr" style="unicode-bidi: isolate;">params</span>.</strong> 
هذه هي الطريقة التي نمرّر بها متغيرات عامة من 
<span dir="ltr" style="unicode-bidi: isolate;">Snakefile</span> 
إلى السكربت الذي نريد تشغيله. هذا أمر بالغ الأهمية لأنه يمنع تكرار المعلومات أو تضاربها. في هذا المثال، نريد تحديد مجلد التنزيل (حيث سيتم تخزين بيانات 
<span dir="ltr" style="unicode-bidi: isolate;">MCD12Q1</span>) في مكان واحد فقط.
</p>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F3C1; تحدٍّ: كتابة سكربت التنزيل
</h3>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تتطلب قاعدة 
<span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> 
أعلاه وجود سكربت 
<span dir="ltr" style="unicode-bidi: isolate;">Python</span> 
باسم 
<span dir="ltr" style="unicode-bidi: isolate;">`download_data.py`</span>. 
اكتب هذا السكربت اعتمادًا على الخطوات التي اتبعناها أعلاه لتنزيل بيانات الغطاء الأرضي.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>
في سكربت 
<span dir="ltr" style="unicode-bidi: isolate;">Python</span> 
يمكنك الوصول إلى 
<span dir="ltr" style="unicode-bidi: isolate;">`params`</span>.
</strong> 
كل زوج 
<span dir="ltr" style="unicode-bidi: isolate;">(key-value)</span> 
نضيفه داخل 
<span dir="ltr" style="unicode-bidi: isolate;">`params`</span> 
سيكون متاحًا داخل السكربت من خلال 
<span dir="ltr" style="unicode-bidi: isolate;">snakemake.params</span>. 
على سبيل المثال، عندما نريد تنزيل البيانات إلى مجلد محدد، يمكننا كتابة:
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>earthaccess.download(results, snakemake.params.dir_mcd12q1)</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<a href="https://github.com/OpenClimateScience/M5-Open-Science-for-Crops/blob/main/notebooks/workflows/download_data_v1.py" dir="ltr">
قارن نسختك من `download_data.py` بالسكربت الخاص بنا
</a>.
</p>

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F6A9; <span style="color:red">انتبه</span>
</h3>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يتم <span dir="ltr" style="unicode-bidi: isolate;">snakemake.params</span> 
<em>حقنه</em> داخل شيفرة 
<span dir="ltr" style="unicode-bidi: isolate;">Python</span> 
عند تنفيذ 
<span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> 
للسكربت. 
<strong>
لا تضف 
<span dir="ltr" style="unicode-bidi: isolate;">`import snakemake`</span> 
إلى سكربتك.
</strong> 
لن يقوم ذلك بما تتوقعه، وسيسبب خطأ.
</p>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تنفيذ أول قاعدة لنا
</h3>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لتشغيل القاعدة 
<span dir="ltr" style="unicode-bidi: isolate;">download_data</span>، 
يجب أن نكون داخل المجلد نفسه الذي يحتوي على ملف 
<span dir="ltr" style="unicode-bidi: isolate;">Snakefile</span>.
</p>

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1FA9F; &#x1F34E; &#x1F427; جميع أنظمة التشغيل
</h3>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لتنفيذ القاعدة:
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>snakemake --cores=1 download_data</code></pre>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<hr/>

<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
إضافة مهمة لمعالجة البيانات
</h2>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
الخطوة التالية هي معالجة بيانات الغطاء الأرضي. لنكتب قاعدة (rule) جديدة في ملف 
<span dir="ltr" style="unicode-bidi: isolate;">Snakefile</span>:
</p>

<pre dir="ltr"><code>
rule process_land_cover:
    input:
        "data/MCD12Q1/MCD12Q1.A2023001.h18v05.061.2024252125305.hdf"
    output:
        "data/processed/MODIS_MCD12Q1_Type5_cereal_croplands_h18v05_2023.tiff"
    script:
        "workflows/process_land_cover.py"
</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
من المفترض أن يكون معنى هذه القاعدة واضحًا بذاته. لمعالجة بيانات الغطاء الأرضي لدينا ملف إدخال محدد يتم تحويله إلى ملف إخراج محدد اعتمادًا على برنامج 
<span dir="ltr" style="unicode-bidi: isolate;">Python</span> معيّن.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F449; ألقِ نظرة على ملف 
<span dir="ltr" style="unicode-bidi: isolate;">process_land_cover.py</span> 
الذي كتبناه. لقد جمعنا فيه جميع الشيفرات التي أنشأناها سابقًا ووضعناها في سكربت واحد.
</p>

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
ملفات الإدخال والإخراج
</h3>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لاحظ أن قاعدتنا الجديدة تحتوي على الكلمتين المفتاحيتين 
<span dir="ltr" style="unicode-bidi: isolate;">input</span> 
و<span dir="ltr" style="unicode-bidi: isolate;">output</span>. 
ولهما معنى خاص في 
<span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span>:
</p>

<ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لا يمكن تنفيذ القاعدة إذا لم تكن جميع ملفات الإدخال المحددة في 
<span dir="ltr" style="unicode-bidi: isolate;">input</span> موجودة.
</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لن تُنفَّذ القاعدة إذا كانت جميع ملفات الإخراج المحددة في 
<span dir="ltr" style="unicode-bidi: isolate;">output</span> موجودة بالفعل. 
يقوم 
<span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> 
بتنفيذ القواعد فقط عندما تكون ملفات الهدف (ملفات الإخراج) غير موجودة بعد.
</li>
</ul>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
داخل ملف 
<span dir="ltr" style="unicode-bidi: isolate;">process_land_cover.py</span> 
يمكننا الوصول إلى ملفات الإدخال والإخراج عبر:
<span dir="ltr" style="unicode-bidi: isolate;">snakemake.input</span> 
و<span dir="ltr" style="unicode-bidi: isolate;">snakemake.output</span>.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
على سبيل المثال، لقراءة ملف الإدخال:
</p>

<pre dir="ltr"><code>
hdf = py4eos.read_file(snakemake.input[0], platform = 'MODIS')
</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
نكتب 
<span dir="ltr" style="unicode-bidi: isolate;">snakemake.input[0]</span> 
لأنه يمكن أن يكون هناك أكثر من ملف إدخال.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
وعند إنشاء ملف الإخراج:
</p>

<pre dir="ltr"><code>
output_raster = rasterio.open(
    snakemake.output[0], 'w+', count = 1, width = width, height = height,
    dtype = np.uint8, crs = pyproj.CRS(6933), transform = new_transform)
</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
في كلتا الحالتين نصل إلى العنصر الأول (والوحيد هنا) من قائمة الإدخال أو الإخراج.
</p>

<h4 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F6A9; انتبه
</h4>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
المتغيران 
<span dir="ltr" style="unicode-bidi: isolate;">snakemake.input</span> 
و<span dir="ltr" style="unicode-bidi: isolate;">snakemake.output</span> 
يتم حقنهما داخل الشيفرة عند تنفيذ القاعدة بواسطة 
<span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span>. 
<strong>لا تقم بإضافة</strong> 
<span dir="ltr" style="unicode-bidi: isolate;">import snakemake</span> 
داخل السكربت — سيؤدي ذلك إلى خطأ.
</p>

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
المهام التي أُنجزت مسبقًا
</h3>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لتنفيذ القاعدة:
</p>

<pre dir="ltr"><code>
snakemake --cores=1 process_land_cover
</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
ماذا يحدث إذا حاولت تنفيذ القاعدة مرة أخرى بعد نجاحها؟
ستظهر رسالة مشابهة لما يلي:
</p>

<pre dir="ltr"><code>
Assuming unrestricted shared filesystem usage.
host: Gullveig
Building DAG of jobs...
Nothing to be done (all requested files are present and up to date).
</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
هذا يعني أن جميع ملفات الإخراج موجودة بالفعل، وبالتالي لا يوجد ما يجب تنفيذه. هذا السلوك منطقي، لأننا نحتاج إنشاء كل ملف مرة واحدة فقط.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
إذا أردت إعادة تنفيذ القاعدة بغض النظر عن وجود ملفات الإخراج، يمكنك استخدام الخيار:
</p>

<pre dir="ltr"><code>
snakemake --force --cores=1 process_land_cover
</code></pre>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<hr>

<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
إضافة مهمة لمعالجة البيانات
</h2>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
الخطوة التالية هي معالجة بيانات الغطاء الأرضي. لنكتب قاعدة جديدة في ملف 
<span dir="ltr" style="unicode-bidi: isolate;">Snakefile</span>:
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>rule process_land_cover:
    input:
        "data/MCD12Q1/MCD12Q1.A2023001.h18v05.061.2024252125305.hdf"
    output:
        "data/processed/MODIS_MCD12Q1_Type5_cereal_croplands_h18v05_2023.tiff"
    script:
        "workflows/process_land_cover.py"</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يفترض أن يكون معنى هذه القاعدة واضحًا: لدينا ملف إدخال محدد يتم تحويله إلى ملف إخراج محدد باستخدام سكربت 
<span dir="ltr" style="unicode-bidi: isolate;">Python</span> 
معرّف بالاسم.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F449; 
<a href="https://github.com/OpenClimateScience/M5-Open-Science-for-Crops/blob/main/notebooks/workflows/process_land_cover.py" dir="ltr">
ألقِ نظرة على سكربت process_land_cover.py الذي كتبناه
</a>. 
لقد جمعنا كل الشيفرة التي كتبناها سابقًا ووضعناها داخل سكربت واحد.
</p>

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
ملفات الإدخال والإخراج
</h3>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لاحظ أن القاعدة الجديدة تحتوي على الكلمتين المفتاحيتين 
<span dir="ltr" style="unicode-bidi: isolate;">input</span> 
و<span dir="ltr" style="unicode-bidi: isolate;">output</span>. 
لهاتين الكلمتين معنى خاص في 
<span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span>.
</p>

<ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لا يمكن تنفيذ القاعدة إذا لم يكن أيٌّ من الملفات المحددة في 
<span dir="ltr" style="unicode-bidi: isolate;">input</span> 
موجودًا.
</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لن تُنفَّذ القاعدة إذا كانت جميع الملفات المحددة في 
<span dir="ltr" style="unicode-bidi: isolate;">output</span> 
موجودة بالفعل. ينفّذ 
<span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> 
فقط القواعد التي لديها مخرجات لم يتم إنشاؤها بعد.
</li>
</ul>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
داخل 
<span dir="ltr" style="unicode-bidi: isolate;">process_land_cover.py</span>، 
يمكننا الوصول إلى ملفات الإدخال والإخراج عبر 
<span dir="ltr" style="unicode-bidi: isolate;">snakemake.input</span> 
و<span dir="ltr" style="unicode-bidi: isolate;">snakemake.output</span>. 
على سبيل المثال، لقراءة ملف الإدخال يمكننا كتابة:
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>hdf = py4eos.read_file(snakemake.input[0], platform='MODIS')</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>
نكتب 
<span dir="ltr" style="unicode-bidi: isolate;">snakemake.input[0]</span> 
لأنه قد يكون هناك أكثر من ملف إدخال.
</strong> 
وبالمثل قد يكون هناك أكثر من ملف إخراج. عند إنشاء الرستر الناتج يمكننا كتابة:
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>output_raster = rasterio.open(
    snakemake.output[0], 'w+', count=1, width=width, height=height,
    dtype=np.uint8, crs=pyproj.CRS(6933), transform=new_transform)</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
في كلتا الحالتين، نصل إلى العنصر الأول (والوحيد) من قائمة 
<span dir="ltr" style="unicode-bidi: isolate;">input</span> 
أو 
<span dir="ltr" style="unicode-bidi: isolate;">output</span>.
</p>

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F6A9; <span style="color:red">انتبه</span>
</h3>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يتم حقن 
<span dir="ltr" style="unicode-bidi: isolate;">snakemake.input</span> 
و<span dir="ltr" style="unicode-bidi: isolate;">snakemake.output</span> 
داخل شيفرة 
<span dir="ltr" style="unicode-bidi: isolate;">Python</span> 
عند تنفيذ السكربت بواسطة 
<span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span>. 
<strong>
لا تضف 
<span dir="ltr" style="unicode-bidi: isolate;">import snakemake</span> 
إلى سكربتك.
</strong> 
سيؤدي ذلك إلى حدوث خطأ.
</p>

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
المهام التي تم تنفيذها مسبقًا
</h3>

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1FA9F; &#x1F34E; &#x1F427; جميع أنظمة التشغيل
</h3>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لتنفيذ القاعدة:
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>snakemake --cores=1 process_land_cover</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>
ماذا يحدث إذا حاولت تنفيذ القاعدة مرة أخرى بعد التنفيذ الأول؟
</strong> 
ينبغي أن ترى رسالة مشابهة لما يلي:
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>Assuming unrestricted shared filesystem usage.
host: Gullveig
Building DAG of jobs...
Nothing to be done (all requested files are present and up to date).</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
هنا يُخبرنا 
<span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> 
أن جميع الملفات المحددة في 
<span dir="ltr" style="unicode-bidi: isolate;">output</span> 
موجودة بالفعل، وبالتالي لا يوجد شيء جديد ليتم تنفيذه. هذا سلوك منطقي، لأننا نحتاج إلى إنشاء كل ملف إخراج مرة واحدة فقط.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
إذا أردت إعادة تنفيذ قاعدة أو أكثر بغض النظر عن وجود ملفات الإخراج، يمكنك إضافة الخيار 
<span dir="ltr" style="unicode-bidi: isolate;">--force</span>:
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>snakemake --force --cores=1 process_land_cover</code></pre>

</div>


In [ ]:
results = earthaccess.search_data(
    short_name = 'VNP16A2GF',
    temporal = TIME_PERIOD,
    bounding_box = tuple(bbox))

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
مثل العديد من منتجات بيانات 
<span dir="ltr" style="unicode-bidi: isolate;">MODIS</span> 
و<span dir="ltr" style="unicode-bidi: isolate;">VIIRS</span>، 
يأتي المنتج 
<span dir="ltr" style="unicode-bidi: isolate;">VNP16A2</span> 
على هيئة مركّبات زمنية لمدة 
<span dir="ltr" style="unicode-bidi: isolate;">8</span> 
أيام. يوجد 
<span dir="ltr" style="unicode-bidi: isolate;">46</span> 
فترة من فترات 
<span dir="ltr" style="unicode-bidi: isolate;">8</span> 
أيام في السنة الواحدة، وتمتد الفترة الزمنية لدراستنا 
<span dir="ltr" style="unicode-bidi: isolate;">TIME_PERIOD</span> 
على سنة واحدة.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لكن عند التحقق من طول 
<span dir="ltr" style="unicode-bidi: isolate;">results</span>، 
نلاحظ وجود 
<span dir="ltr" style="unicode-bidi: isolate;">47</span> 
حبيبة بيانات 
<span dir="ltr" style="unicode-bidi: isolate;">(granules)</span>. 
وذلك لأن النتائج تتضمن أيضًا الفترة الأخيرة ذات 
<span dir="ltr" style="unicode-bidi: isolate;">8</span> 
أيام، والتي تبدأ في اليوم الأخير من سلسلتنا الزمنية.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
بوجه عام، من الأفضل دائمًا التحقق من طول النتائج التي تعيدها الدالة 
<span dir="ltr" style="unicode-bidi: isolate;">earthaccess.search_data()</span>.
</p>

</div>


In [ ]:
# Check that we're getting the expected number of granules
len(results)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
كما فعلنا سابقًا، سنقوم بتنزيل البيانات فقط إذا لم تكن قد نُزِّلت من قبل.
</p>

</div>


In [ ]:
# Only download the files once; i.e., if we haven't already downloaded any
if len(glob.glob(f'{VNP16_DIR}/*')) == len(results):
    earthaccess.download(results, VNP16_DIR)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
وسنستخدم مكتبة 
<span dir="ltr" style="unicode-bidi: isolate;">py4eos</span> 
مرة أخرى لقراءة الملف، على الرغم من أنه ملف بصيغة 
<span dir="ltr" style="unicode-bidi: isolate;">HDF5</span>، 
لأننا نحتاج إلى طريقة سهلة لفتح البيانات كبيانات نقطية 
<span dir="ltr" style="unicode-bidi: isolate;">(raster dataset)</span>.
</p>

</div>


In [ ]:
hdf = py4eos.read_file('data/VNP16A2/VNP16A2GF.A2024001.h18v05.002.2025021191652.h5', platform = 'VIIRS')
hdf

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يحتوي الملف على مجموعتي بيانات نهتم بهما:
</p>

<ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<span dir="ltr" style="unicode-bidi: isolate;">PET_500m</span> 
تشير إلى بيانات 
<span dir="ltr" style="unicode-bidi: isolate;">PET</span>.
</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<span dir="ltr" style="unicode-bidi: isolate;">ET_500m</span> 
تشير إلى بيانات 
<span dir="ltr" style="unicode-bidi: isolate;">ET</span> 
(وسنستخدمها لاحقًا).
</li>
</ul>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
استنادًا إلى دليل المستخدم 
(<a href="https://dx.doi.org/10.5067/VIIRS/VNP16A2GF.002" dir="ltr">المتاح هنا</a>)، 
نعلم أن كلتا مجموعتي البيانات لهما نطاق صالح للقيم، كما أنهما مُقاسَتان 
<span dir="ltr" style="unicode-bidi: isolate;">(scaled)</span> 
لتقليل حجم الملف.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
نستخدم الدالة 
<span dir="ltr" style="unicode-bidi: isolate;">numpy.where()</span> 
لإسناد القيمة 
<span dir="ltr" style="unicode-bidi: isolate;">np.nan</span> 
إلى القيم الواقعة خارج النطاق الصالح، ثم نعيد مقياس القيم الصالحة بضربها في العامل 
<span dir="ltr" style="unicode-bidi: isolate;">0.1</span>.
</p>

</div>


In [ ]:
# Values greater than or equal to 32700 are NoData values
pet0 = hdf.get('PET_500m')
pet = np.where(np.abs(pet0) >= 32700, np.nan, pet0 * 0.1)

In [ ]:
pyplot.imshow(pet, interpolation = 'nearest')
pyplot.colorbar()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F3C1; تحدٍّ: تحديث سكربت التنزيل
</h3>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
حدّث القاعدة 
<span dir="ltr" style="unicode-bidi: isolate;">download_data</span> 
وكذلك السكربت 
<span dir="ltr" style="unicode-bidi: isolate;">download_data.py</span> 
بحيث يقومان أيضًا بتنزيل بيانات 
<span dir="ltr" style="unicode-bidi: isolate;">VIIRS VNP16</span> 
للنتح-التبخر 
<span dir="ltr" style="unicode-bidi: isolate;">ET</span> 
إلى المجلد الصحيح، استنادًا إلى المثال أعلاه.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<a href="https://github.com/OpenClimateScience/M5-Open-Science-for-Crops/blob/main/notebooks/workflows/download_data_v2.py" dir="ltr">
قارن نسختك من `download_data.py` بالسكربت الخاص بنا
</a>. 
لاحظ بشكل خاص أننا أصبح لدينا عدة معاملات 
<span dir="ltr" style="unicode-bidi: isolate;">params</span>:
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>rule download_data:
    params:
        dir_mcd12q1="data/MCD12Q1",
        dir_vnp16a2="data/VNP16A2"</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F449; لاحظ أن الفاصلة ضرورية عند وجود أكثر من عنصر ضمن 
<span dir="ltr" style="unicode-bidi: isolate;">params</span>.
</p>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<hr>

<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
إعادة أخذ العينات وإسقاط بيانات ET و PET
</h2>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
كما فعلنا مع بيانات الغطاء الأرضي، نريد إسقاط بيانات 
<span dir="ltr" style="unicode-bidi: isolate;">ET</span> 
و<span dir="ltr" style="unicode-bidi: isolate;">PET</span> 
على شبكة 
<span dir="ltr" style="unicode-bidi: isolate;">EASE-Grid 2.0</span> 
بدقة 
<span dir="ltr" style="unicode-bidi: isolate;">1</span> 
كم. غير أن الأمر هنا أكثر تعقيدًا قليلًا، إذ لا توجد قيمة واحدة فقط تمثل 
<span dir="ltr" style="unicode-bidi: isolate;">NoData</span>؛ بل يجب اعتبار جميع القيم الأكبر من 
<span dir="ltr" style="unicode-bidi: isolate;">32700</span> 
قيمًا غير صالحة.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لذلك سنحتاج أولًا إلى استخدام الدالة 
<span dir="ltr" style="unicode-bidi: isolate;">read()</span> 
من كائن 
<span dir="ltr" style="unicode-bidi: isolate;">rasterio</span> 
لاستخراج مصفوفة 
<span dir="ltr" style="unicode-bidi: isolate;">numpy</span> 
بعد إعادة أخذ العينات إلى دقة 
<span dir="ltr" style="unicode-bidi: isolate;">1</span> 
كم، ثم كتابة هذه المصفوفة في كائن 
<span dir="ltr" style="unicode-bidi: isolate;">rasterio</span> 
جديد قبل تنفيذ عملية الإسقاط.
</p>

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F3C1; تحدٍّ
</h3>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>
اكتب دالة تقوم بإعادة أخذ العينات ثم إسقاط بيانات 
<span dir="ltr" style="unicode-bidi: isolate;">VIIRS ET</span> 
(أو 
<span dir="ltr" style="unicode-bidi: isolate;">PET</span>) 
على شبكة 
<span dir="ltr" style="unicode-bidi: isolate;">EASE-Grid 2.0</span> 
بدقة 
<span dir="ltr" style="unicode-bidi: isolate;">1</span> 
كم.
</strong> 
استخدم ترويسة الدالة التالية كنقطة بداية:
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>def reproject_viirs(hdf, field, output_path = '', driver = 'MEM'):
    '''
    Reprojects a VIIRS ET dataset to the global EASE-Grid 2.0.

    Parameters
    ----------
    hdf : py4eos.EOSHDF4
        The EOSHDF4 instance connected to the VIIRS ET dataset
    field : str
        The name of the data variable, e.g., "ET_500m"
    output_path : str
        (Optional) The file path, if writing to a file on disk
    driver : str
        (Optional) The driver name, defaults to "MEM"

    Returns
    -------
    rasterio.io.DatasetWriter
    '''
    ...</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
هذه مسألة متقدمة نسبيًا! اطّلع على الحل المقترح أدناه بعد أن تحاول تنفيذها بنفسك.
</p>

</div>


In [ ]:
def reproject_viirs(hdf, field, output_path = '', driver = 'MEM'):
    '''
    Reprojects a VIIRS ET dataset to the global EASE-Grid 2.0.

    Parameters
    ----------
    hdf : py4eos.EOSHDF4
        The EOSHDF4 instance connected to the VIIRS ET dataset
    field : str
        The name of the data variable, e.g., "ET_500m"
    output_path : str
        (Optional) The file path, if writing to a file on disk
    driver : str
        (Optional) The driver name, defaults to "MEM"

    Returns
    -------
    rasterio.io.DatasetWriter
    '''
    et_raster = hdf.to_rasterio(
        field, filename = '', driver = 'MEM', nodata = 32766., scale_and_offset = True)
    
    # First, resample the ET data to 1-km resolution
    arr = et_raster.read(out_shape = (1200, 1200), resampling = Resampling.average)
    arr = np.where(np.abs(arr) >= 32700, np.nan, arr)
    # We have to re-create the raster dataset, now at 1-km resolution
    et_raster_1km = rasterio.open(
        '', 'w+', driver = 'MEM', height = 1200, width = 1200,
        count = 1, dtype = np.float32, crs = et_raster.crs, 
        transform = et_raster.transform * et_raster.transform.scale(2)) # NOTE: Scaling to 1 km
    et_raster_1km.write(arr[0], 1)
    
    # Second, project the data onto a global EASE-Grid 2.0
    new_transform, width, height = calculate_default_transform(
        et_raster_1km.crs, pyproj.CRS(6933), 1200, 1200, *et_raster_1km.bounds)
    et_raster_ease2 = rasterio.open(
        output_path, 'w+', driver = driver, height = height, width = width,
        count = 1, dtype = np.float32, crs = pyproj.CRS(6933), transform = new_transform)
    reproject(
        source = rasterio.band(et_raster_1km, 1),
        destination = rasterio.band(et_raster_ease2, 1),
        resampling = Resampling.bilinear,
        src_nodata = np.nan, # Necessary so that missing data is interpolated
        dst_nodata = np.nan)
    return et_raster_ease2

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
دعونا نستعرض بيانات ET من الملف الذي قمنا بفتحه سابقًا.
</div>

In [ ]:
et_raster_ease2 = reproject_viirs(hdf, 'ET_500m')
img = et_raster_ease2.read(1)
pyplot.imshow(img, interpolation = 'nearest')
pyplot.colorbar()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يبدو جيدًا! الآن نحتاج فقط إلى تطبيق هذه الدالة على كل ملف قمنا بتنزيله. يمكننا تنفيذ ذلك باستخدام حلقة 
<span dir="ltr" style="unicode-bidi: isolate;">for</span>. 
وسنستخدم الدالة 
<span dir="ltr" style="unicode-bidi: isolate;">tqdm()</span> 
من الحزمة 
<span dir="ltr" style="unicode-bidi: isolate;">tqdm</span> 
لعرض شريط تقدم أثناء التنفيذ.
</p>

</div>


In [ ]:
from tqdm import tqdm

# Get a list of all the files
file_list = glob.glob(f'{VNP16_DIR}/*')
file_list.sort()

et_data = []
pet_data = []
for filename in tqdm(file_list):
    # Read the date from the filename
    date = datetime.datetime.strptime(filename.split('/')[-1].split('.')[1], 'A%Y%j')
    # Convert the date to a YYYYMMDD string
    date_str = date.strftime('%Y%m%d')
    # Read in the input VNP16 granule
    hdf = py4eos.read_file(filename, platform = 'VIIRS')
    # Prepare the output filename, project the raster data
    et_filename = f'{OUTPUT_VNP16_DIR}/VNP16_ET_mm_8day-1_{date_str}.tiff'
    et0 = reproject_viirs(hdf, 'ET_500m', et_filename, driver = 'GTiff')
    pet_filename = f'{OUTPUT_VNP16_DIR}/VNP16_PET_mm_8day-1_{date_str}.tiff'
    pet0 = reproject_viirs(hdf, 'PET_500m', pet_filename, driver = 'GTiff')

    # Open the GeoTIFF files as xarray Datasets
    et = rioxarray.open_rasterio(et_filename)\
        .rename(band = 'time')\
        .assign_coords(time = [date])
    et_data.append(et)
    pet = rioxarray.open_rasterio(pet_filename)\
        .rename(band = 'time')\
        .assign_coords(time = [date])
    pet_data.append(pet)

# Convert from [mm 8day-1] to [mm day-1]
ds_et0 = xr.concat(et_data, dim = 'time').to_dataset(name = 'ET') / 8.0
ds_pet0 = xr.concat(pet_data, dim = 'time').to_dataset(name = 'PET') / 8.0
ds_et = xr.merge([ds_et0, ds_pet0])
ds_et.to_netcdf('data/processed/VNP16_ET_and_PET_mm_day-1.nc4')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F3AF; أفضل الممارسات
</h3>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لاحظ أننا استخدمنا 
<span dir="ltr" style="unicode-bidi: isolate;">file_list.sort()</span> 
أعلاه. لماذا؟
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
قد لا يكون لذلك تأثير كبير في هذا المثال تحديدًا، ولكن بشكل عام 
<strong>
من المهم جدًا التأكد من أن قائمة الملفات مرتبة بالترتيب الصحيح قبل البدء في معالجتها.
</strong>
إذا كانت الملفات تمثل سلسلة زمنية، فقد يكون من الضروري معالجتها بترتيب زمني.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
إذا كانت أسماء الملفات تتضمن تاريخًا منسقًا بترتيب سنة-شهر-يوم 
<span dir="ltr" style="unicode-bidi: isolate;">(YYYYMMDD)</span>، 
وهو من أفضل الممارسات أيضًا، فإن استدعاء الدالة 
<span dir="ltr" style="unicode-bidi: isolate;">sort()</span> 
على قائمة الملفات سيضمن ترتيبها زمنيًا.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>
للتأكيد: عندما يكون التاريخ في اسم الملف منسقًا بترتيب سنة-شهر-يوم 
<span dir="ltr" style="unicode-bidi: isolate;">(YYYYMMDD)</span>، 
فإن الفرز الأبجدي العددي باستخدام الدالة 
<span dir="ltr" style="unicode-bidi: isolate;">sort()</span> 
يعادل الفرز الزمني تمامًا.
</strong>
</p>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<hr>

<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
إضافة معالجة بيانات ET إلى <span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span>
</h2>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
نحتاج الآن إلى قاعدة جديدة في 
<span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> 
لمعالجة بيانات 
<span dir="ltr" style="unicode-bidi: isolate;">ET</span>. 
سابقًا حدّدنا مجلد بيانات 
<span dir="ltr" style="unicode-bidi: isolate;">VIIRS VNP16</span> 
باستخدام الكلمة المفتاحية 
<span dir="ltr" style="unicode-bidi: isolate;">params</span> 
داخل قاعدة 
<span dir="ltr" style="unicode-bidi: isolate;">download_data</span>. 
لكن هناك عدة قواعد تحتاج إلى معرفة مكان تخزين البيانات! فهل يمكننا تجنّب تكرار 
<span dir="ltr" style="unicode-bidi: isolate;">params</span> 
نفسها في عدة قواعد؟
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
نعم، هناك على الأقل طريقتان للتعامل مع ذلك.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
أولًا، بما أن كل 
<span dir="ltr" style="unicode-bidi: isolate;">Snakefile</span> 
هو في الأساس امتداد لشيفرة 
<span dir="ltr" style="unicode-bidi: isolate;">Python</span>، 
يمكننا تعريف متغيرات عامة في أعلى الملف ثم استخدامها داخل القواعد:
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>DIR_MCD12Q1 = "data/MCD12Q1"
DIR_VNP16A2 = "data/VNP16A2"

rule download_data:
    params:
        dir_mcd12q1=DIR_MCD12Q1,
        dir_vnp16a2=DIR_VNP16A2
    script:
        "workflows/download_data.py"</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>
لكن سيكون من الأسهل كثيرًا إنشاء ملف إعدادات 
<span dir="ltr" style="unicode-bidi: isolate;">configuration file</span> 
لحفظ المتغيرات العامة.
</strong>
ملف الإعدادات هو ببساطة ملف بصيغة 
<span dir="ltr" style="unicode-bidi: isolate;">YAML</span> 
يحتوي على أزواج مفتاح-قيمة. يمكننا تخزين المتغيرات العامة وأي معلومات أخرى تحدد سلوك سير العمل. ينبغي أن يبدو ملف الإعدادات الأولي كما يلي:
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>inputs:
  mcd12q1: "data/MCD12Q1"
  vnp16a2: "data/VNP16A2"</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
بعد ذلك، كل ما علينا فعله هو إضافة ملف الإعدادات إلى 
<span dir="ltr" style="unicode-bidi: isolate;">Snakefile</span>. 
يمكننا كتابة القاعدة الجديدة لمعالجة بيانات 
<span dir="ltr" style="unicode-bidi: isolate;">ET</span> 
وإعادة كتابة قاعدة التنزيل كما يلي:
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>configfile: "config.yaml"

rule download_data:
    script:
        "workflows/download_data.py"
        
rule process_et:
    output:
        "data/processed/VNP16_ET_and_PET_mm_day-1.nc4"
    script:
        "workflows/process_et.py"</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
بدلًا من الوصول إلى 
<span dir="ltr" style="unicode-bidi: isolate;">snakemake.params</span> 
داخل السكربتات، سنستخدم الآن 
<span dir="ltr" style="unicode-bidi: isolate;">snakemake.config</span>. 
على سبيل المثال، في 
<span dir="ltr" style="unicode-bidi: isolate;">download_data.py</span> 
يمكننا كتابة:
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>earthaccess.download(
    results_mcd12q1,
    os.path.dirname(snakemake.config['inputs']['mcd12q1'])
)</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F449; 
<a href="https://github.com/OpenClimateScience/M5-Open-Science-for-Crops/blob/main/notebooks/workflows/process_et_v1.py" dir="ltr">
اطّلع على ما كتبناه في process_et.py
</a>.
</p>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<hr>

<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
الحصول على بيانات الهطول من IMERG
</h2>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
بعد ذلك، لنقم بتنزيل بيانات الهطول من منتج 
<span dir="ltr" style="unicode-bidi: isolate;">IMERG-Final</span> 
التابع لمهمة 
<span dir="ltr" style="unicode-bidi: isolate;">GPM</span> 
(<a href="https://dx.doi.org/10.5067/GPM/IMERGDF/DAY/07" dir="ltr">الرابط</a>).
</p>

</div>


In [ ]:
results = earthaccess.search_data(
    short_name = 'GPM_3IMERGDF',
    temporal = TIME_PERIOD)

In [ ]:
# Only download the files once; i.e., if we haven't already downloaded any
if len(glob.glob(f'{IMERG_DIR}/*')) == 0:
    earthaccess.download(results, IMERG_DIR)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
بيانات 
<span dir="ltr" style="unicode-bidi: isolate;">IMERG-Final</span> 
متوفرة بصيغة 
<span dir="ltr" style="unicode-bidi: isolate;">netCDF4</span>، 
والتي—كما ناقشنا في الوحدة 
<span dir="ltr" style="unicode-bidi: isolate;">2</span> 
(<a href="https://github.com/OpenClimateScience/M2-Computational-Climate-Science" dir="ltr">الرابط</a>)—يمكن التعامل معها بسهولة أكبر في 
<span dir="ltr" style="unicode-bidi: isolate;">Python</span> 
باستخدام مكتبة 
<span dir="ltr" style="unicode-bidi: isolate;">xarray</span>.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تختلف بيانات 
<span dir="ltr" style="unicode-bidi: isolate;">IMERG-Final</span> 
عن بيانات الغطاء الأرضي 
<span dir="ltr" style="unicode-bidi: isolate;">MODIS</span> 
وبيانات 
<span dir="ltr" style="unicode-bidi: isolate;">VIIRS ET</span> 
في أن كل ملف يمثل الكرة الأرضية كاملة. لذلك سنحتاج إلى قصّ كل ملف ليقتصر على منطقة دراستنا.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
إحدى الطرق السهلة للقيام بذلك هي استخدام الخاصية 
<span dir="ltr" style="unicode-bidi: isolate;">bounds</span> 
من كائن 
<span dir="ltr" style="unicode-bidi: isolate;">rasterio</span> 
الذي أنشأناه سابقًا. نستخدم 
<span dir="ltr" style="unicode-bidi: isolate;">shapely.geometry.Polygon</span> 
لتمثيل صندوق الإحاطة 
<span dir="ltr" style="unicode-bidi: isolate;">(bounding box)</span> 
بصيغة يمكن أن تفهمها حزمة 
<span dir="ltr" style="unicode-bidi: isolate;">rioxarray</span>.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تذكّر أن 
<span dir="ltr" style="unicode-bidi: isolate;">rioxarray</span> 
هي حزمة مبنية فوق 
<span dir="ltr" style="unicode-bidi: isolate;">xarray</span> 
وتسهّل العمل مع البيانات المكانية ضمن بيئة 
<span dir="ltr" style="unicode-bidi: isolate;">xarray</span>.
</p>

</div>


In [ ]:
# Getting the bounds of our VIIRS tile, for clipping other datasets
bb = et_raster_ease2.bounds
bounds = Polygon([
    (bb.left, bb.bottom), 
    (bb.left, bb.top),
    (bb.right, bb.top),
    (bb.right, bb.bottom)
])
bounds

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لاحظ أن إحداثيات صندوق الإحاطة لدينا معبَّر عنها بالمتر شرقًا 
<span dir="ltr" style="unicode-bidi: isolate;">(meters easting)</span> 
والمتر شمالًا 
<span dir="ltr" style="unicode-bidi: isolate;">(meters northing)</span> 
(كما هو موضح أدناه). ويرجع ذلك إلى أن نظام الإحداثيات المرجعي 
<span dir="ltr" style="unicode-bidi: isolate;">(CRS)</span> 
لبيانات 
<span dir="ltr" style="unicode-bidi: isolate;">ET</span> 
أصبح الآن شبكة عالمية 
<span dir="ltr" style="unicode-bidi: isolate;">EASE-Grid 2.0</span>.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لذلك سنحتاج إلى التأكد من إسقاط بيانات الهطول أيضًا على نظام الإحداثيات المرجعي نفسه 
<span dir="ltr" style="unicode-bidi: isolate;">CRS</span>.
</p>

</div>


In [ ]:
bb

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يمكننا أيضًا حفظ هندسة صندوق الإحاطة هذا في ملف لاستخدامه لاحقًا.
</p>

</div>


In [ ]:
import shapely
import json

with open('data/processed/bounds.geojson', 'w') as file:
    json.dump(shapely.to_geojson(bounds), file)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تذكّر أنه عند تثبيت حزمة 
<span dir="ltr" style="unicode-bidi: isolate;">rioxarray</span>، 
يمكننا الوصول إلى الخاصية 
<span dir="ltr" style="unicode-bidi: isolate;">.rio</span> 
لأي كائن من نوع 
<span dir="ltr" style="unicode-bidi: isolate;">xarray</span> 
لاستخدام الدوال التي تعمل على البيانات المكانية. على سبيل المثال…
</p>

</div>


In [ ]:
# Just an example
ds = xr.open_dataset('data/IMERG/3B-DAY.MS.MRG.3IMERG.20231001-S000000-E235959.V07B.nc4')
ds.rio

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
أصبحنا الآن جاهزين لمعالجة بيانات الهطول من منتج 
<span dir="ltr" style="unicode-bidi: isolate;">IMERG-Final</span>. 
وبما أن كل ملف من ملفات 
<span dir="ltr" style="unicode-bidi: isolate;">IMERG-Final</span> 
يمكن فتحه ككائن 
<span dir="ltr" style="unicode-bidi: isolate;">xarray.Dataset</span>، 
فإن المعالجة تتم على شكل سلسلة من الخطوات، حيث تُنفَّذ كل خطوة باستدعاء دالة 
<span dir="ltr" style="unicode-bidi: isolate;">method</span> 
على كائن 
<span dir="ltr" style="unicode-bidi: isolate;">xarray.Dataset</span>.
</p>

<ol style="direction: rtl; unicode-bidi: plaintext; text-align: right; padding-right: 1.2em;">
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
أولًا، نحتاج إلى ترتيب أبعاد كائن 
<span dir="ltr" style="unicode-bidi: isolate;">precipitation</span> 
بحسب الترتيب: الزمن، خط العرض، خط الطول. يمكننا القيام بذلك باستخدام الدالة 
<a href="https://docs.xarray.dev/en/stable/generated/xarray.Dataset.transpose.html" dir="ltr">
transpose()
</a> 
الخاصة بكائن 
<span dir="ltr" style="unicode-bidi: isolate;">Dataset</span>.
</li>

<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
بعد ذلك، نحتاج إلى إسناد نظام الإحداثيات المرجعي 
<span dir="ltr" style="unicode-bidi: isolate;">(CRS)</span> 
وتحديد الأبعاد التي تمثل المحورين 
<span dir="ltr" style="unicode-bidi: isolate;">X</span> 
و<span dir="ltr" style="unicode-bidi: isolate;">Y</span> 
ضمن هذا النظام. نستخدم لذلك الدالتين 
<span dir="ltr" style="unicode-bidi: isolate;">write_crs()</span> 
و<span dir="ltr" style="unicode-bidi: isolate;">set_spatial_dims()</span>. 
لاحظ أننا نضيف السابقة 
<span dir="ltr" style="unicode-bidi: isolate;">.rio</span> 
قبل كل منهما لأن هاتين الدالتين جزء من حزمة 
<span dir="ltr" style="unicode-bidi: isolate;">rioxarray</span>.
</li>

<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
بعد ذلك، نصبح جاهزين لإسقاط البيانات إلى شبكة عالمية 
<span dir="ltr" style="unicode-bidi: isolate;">EASE-Grid 2.0</span> 
برمز 
<span dir="ltr" style="unicode-bidi: isolate;">EPSG 6933</span>. 
ولتقليل حجم الملفات، سنعيد أخذ العينات إلى دقة 
<span dir="ltr" style="unicode-bidi: isolate;">9</span> 
كم. يتم ذلك باستخدام الدالة 
<span dir="ltr" style="unicode-bidi: isolate;">reproject()</span>، 
وهي أيضًا من وظائف 
<span dir="ltr" style="unicode-bidi: isolate;">rioxarray</span>.
</li>

<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
أخيرًا، نقوم بقص البيانات المُسقَطة وفق حدود منطقة الدراسة 
<span dir="ltr" style="unicode-bidi: isolate;">bounds</span> 
باستخدام الدالة 
<span dir="ltr" style="unicode-bidi: isolate;">clip()</span>، 
وهي وظيفة أخرى من 
<span dir="ltr" style="unicode-bidi: isolate;">rioxarray</span>.
</li>
</ol>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لاحظ أننا استخدمنا حلقة 
<span dir="ltr" style="unicode-bidi: isolate;">for</span> 
مرة أخرى لمعالجة كل ملف على حدة. ونقوم بإضافة كل مجموعة بيانات بعد الإسقاط والقص إلى قائمة 
<span dir="ltr" style="unicode-bidi: isolate;">Python</span> 
لتخزينها.
</p>

</div>


In [ ]:
# NOTE: This may take up to 10 minutes to complete for 1 year's worth of data

file_list = glob.glob(f'{IMERG_DIR}/*.nc4')
file_list.sort()

stack = []
for filename in tqdm(file_list):
    ds = xr.open_dataset(filename)
    # Anytime we use the .rio attribute, we must have rioxarray installed
    ds_ease2 = ds[['precipitation']]\
        .transpose('time', 'lat', 'lon')\
        .rio.write_crs(4326)\
        .rio.set_spatial_dims('lon', 'lat')\
        .rio.reproject(pyproj.CRS(6933), resolution = 9000)\
        .rio.clip([bounds])
    stack.append(ds_ease2)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F3AF; أفضل الممارسات
</h3>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لاحظ أننا استدعينا الدالة 
<span dir="ltr" style="unicode-bidi: isolate;">sort()</span> 
مرة أخرى على قائمة الملفات 
<span dir="ltr" style="unicode-bidi: isolate;">file_list</span>. 
في هذا السياق، يُعدّ ذلك أمرًا أساسيًا، لأننا نريد أن تكون مجموعات البيانات التي نضعها في 
<span dir="ltr" style="unicode-bidi: isolate;">stack</span> 
مرتبة ترتيبًا زمنيًا.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>
وكما رأينا سابقًا، يمكننا دمج جميع مجموعات البيانات المنفصلة هذه في كائن واحد من نوع 
<span dir="ltr" style="unicode-bidi: isolate;">xarray.Dataset</span> 
باستخدام الدالة 
<span dir="ltr" style="unicode-bidi: isolate;">concat()</span>.
</strong>
</p>

</div>


In [ ]:
ds_precip = xr.concat(stack, dim = 'time')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<hr>

<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تجهيز المنتجات البيانية المشتقة
</h2>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
الكائن 
<span dir="ltr" style="unicode-bidi: isolate;">ds_precip</span> 
يوجد حاليًا فقط في ذاكرة الحاسوب. لذلك يجب ألا ننسى كتابة مجموعة البيانات إلى القرص!
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يُعدّ ذلك أمرًا سهلًا باستخدام 
<span dir="ltr" style="unicode-bidi: isolate;">xarray</span>، 
وإذا قمنا بحفظ البيانات في ملف بصيغة 
<span dir="ltr" style="unicode-bidi: isolate;">netCDF4</span>، 
فسيتم حفظ جميع السمات 
<span dir="ltr" style="unicode-bidi: isolate;">(attributes)</span> 
والبيانات الوصفية 
<span dir="ltr" style="unicode-bidi: isolate;">(metadata)</span> 
الأخرى تلقائيًا أيضًا.
</p>

</div>


In [ ]:
ds_precip.to_netcdf(f'{OUTPUT_IMERG_DIR}/IMERG_precip_mm_day-1_for_study_area.nc4')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
اخترت أن أضع وحدات الهطول 
<span dir="ltr" style="unicode-bidi: isolate;">(mm per hour)</span> 
في اسم الملف، ولكن إذا كنت قد نسيت ذلك، فلاحظ أن الوحدة مخزنة أيضًا كخاصية 
<span dir="ltr" style="unicode-bidi: isolate;">attribute</span> 
ضمن متغير 
<span dir="ltr" style="unicode-bidi: isolate;">precipitation</span>.
</p>

</div>


In [ ]:
ds_precip.precipitation.units

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<hr>

<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
إضافة معالجة بيانات الهطول إلى <span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span>
</h2>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
نحتاج إلى إضافة ما يلي إلى الملف 
<span dir="ltr" style="unicode-bidi: isolate;">download_data.py</span>:
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code># download_data.py
results_imerg = earthaccess.search_data(
    short_name='GPM_3IMERGDF',
    temporal=TIME_PERIOD)
earthaccess.download(results_imerg, snakemake.config['inputs']['imerg'])</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
كما نحتاج إلى إضافة مجلد بيانات 
<span dir="ltr" style="unicode-bidi: isolate;">IMERG-Final</span> 
إلى ملف الإعدادات:
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code># config.yaml
inputs:
  mcd12q1: "data/MCD12Q1"
  vnp16a2: "data/VNP16A2"
  imerg: "data/IMERG"</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لمعالجة بيانات الهطول، نحتاج إلى معرفة حدود 
<span dir="ltr" style="unicode-bidi: isolate;">bounds</span> 
بيانات 
<span dir="ltr" style="unicode-bidi: isolate;">ET</span> 
بعد إسقاطها. في الأعلى استخرجنا الحدود من أحد ملفات 
<span dir="ltr" style="unicode-bidi: isolate;">ET</span> 
الناتجة؛ لذلك فإن معالجة بيانات الهطول تعتمد على معالجة بيانات 
<span dir="ltr" style="unicode-bidi: isolate;">ET</span> 
و<span dir="ltr" style="unicode-bidi: isolate;">PET</span> 
أولًا.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لربط القواعد بالترتيب الصحيح في 
<span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span>، 
يمكننا جعل قاعدة معالجة الهطول تعتمد على ملف إدخال يمثل أحد مخرجات قاعدة معالجة 
<span dir="ltr" style="unicode-bidi: isolate;">ET</span>:
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code># Snakefile
rule process_et:
    output:
        "data/processed/bounds.geojson",
        "data/processed/VNP16_ET_and_PET_mm_day-1.nc4"
    script:
        "workflows/process_et.py"

rule process_precip:
    input:
        "data/processed/bounds.geojson"
    output:
        "data/processed/IMERG_precip_mm_day-1_for_study_area.nc4"
    script:
        "workflows/process_precip.py"</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
بما أن قاعدة 
<span dir="ltr" style="unicode-bidi: isolate;">process_precip</span> 
تعتمد على الملف 
<span dir="ltr" style="unicode-bidi: isolate;">"data/processed/bounds.geojson"</span> 
كمدخل، فلن يتم تنفيذها إلا إذا كان هذا الملف موجودًا، مما يضمن تنفيذ قاعدة 
<span dir="ltr" style="unicode-bidi: isolate;">process_et</span> 
أولًا.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
في ملف 
<span dir="ltr" style="unicode-bidi: isolate;">process_et.py</span> 
نحتاج بالطبع إلى تصدير ملف 
<span dir="ltr" style="unicode-bidi: isolate;">GeoJSON</span>:
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code># process_et.py
bb = et.bounds
bounds = Polygon([
    (bb.left, bb.bottom),
    (bb.left, bb.top),
    (bb.right, bb.top),
    (bb.right, bb.bottom)
])
with open(snakemake.output[0], 'w') as file:
    json.dump(shapely.to_geojson(bounds), file)</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
وفي ملف 
<span dir="ltr" style="unicode-bidi: isolate;">process_precip.py</span> 
نحتاج إلى قراءة هذا الملف:
</p>

<pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code># process_precip.py
with open(snakemake.input[0], 'r') as file:
    bounds = shapely.from_geojson(json.load(file))</code></pre>

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F6A9; <span style="color:red">انتبه</span>
</h3>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لاحظ أن قاعدة 
<span dir="ltr" style="unicode-bidi: isolate;">process_et</span> 
أصبحت الآن تحتوي على ملفي إخراج. من المهم جدًا التأكد من وجود فاصلة بين اسمي الملفين.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
كما يجب الانتباه في ملف 
<span dir="ltr" style="unicode-bidi: isolate;">process_et.py</span> 
إلى التمييز بين 
<span dir="ltr" style="unicode-bidi: isolate;">snakemake.output[0]</span> 
و<span dir="ltr" style="unicode-bidi: isolate;">snakemake.output[1]</span>.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F449; 
<a href="https://github.com/OpenClimateScience/M5-Open-Science-for-Crops/blob/main/notebooks/workflows/process_precip.py" dir="ltr">
اطّلع على ما كتبناه في process_precip.py
</a>.
</p>

</div>
